In [1]:
with open('input.txt','r',encoding='utf-8') as f:
    text=f.read()

In [2]:
len(text)

1115394

In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [46]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
itos={i:ch for i,ch in enumerate(chars)}
stoi={ch:i for i,ch in itos.items()}
encode=lambda s:[stoi[c] for c in s]
decode=lambda l:''.join(itos[i] for i in l)
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [5]:
import torch
data=encode(text)
data=torch.tensor(data,dtype=torch.long)
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [6]:
n=int(0.9*len(data))
train=data[:n]
test=data[n:]
print(train.shape,test.shape)


torch.Size([1003854]) torch.Size([111540])


In [7]:
torch.randint(10,(3,))

tensor([6, 8, 2])

In [114]:
torch.manual_seed(1337)
batch_size=4
block_size=8

def get_batch(split):
    data=train if split=="train" else test 
    ix=torch.randint(len(data)-block_size,(batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x,y
xb,yb=get_batch(train)

In [115]:
yb

tensor([[ 1, 52, 53, 58,  1, 58, 47, 50],
        [ 1, 54, 50, 39, 52, 58, 43, 58],
        [58, 46, 47, 57,  1, 50, 47, 60],
        [32, 46, 43, 56, 43,  1, 42, 53]])

In [58]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
    def forward(self,idx,targets=None):
        logits=self.token_embedding_table(idx)
        if targets is None:
            loss=None 
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)

        return logits,loss 
    
    def generate(self,idx,max_new_token):
        for _ in range(max_new_token):
            logits,loss=self(idx)
            logits=logits[:,-1,:]
            probs=F.softmax(logits,dim=-1)
            idx.next=torch.multinomial(probs,num_samples=1)
            idx=torch.cat((idx,idx.next),dim=1)
        return idx
    
m=BigramLanguageModel(vocab_size)
logits,loss=m(xb,yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_token=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.4913, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [110]:
optimizer=torch.optim.AdamW(m.parameters(),lr=1e-3)

In [112]:
batch_size=32
for steps in range(1000):
    xb,yb=get_batch("train")

    logits,loss=m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.4635848999023438


In [113]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_token=100)[0].tolist()))


ABud thereanby.
Thyooutondwoty?
Mods:
TE:
LO,
t d s justhilo tenor fomy t, wior!


Gondorr:

TI INow


tensor([[0.]])